# Домашнее задание №1
### 1. Загрузить данные из индивидуальных xls файлов в виде связанных таблиц в базу данных.

In [1]:
import sqlite3
import openpyxl
import datetime
import pandas as pd
import os

In [2]:
conn = sqlite3.connect("data_base.db")

In [3]:
#Создаём пустую таблицу для анкет с необходимыми нам колонками
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Applications (
id_number INTEGER NOT NULL PRIMARY KEY,
date_of_birth DATE NOT NULL,
gender BOOLEAN NOT NULL,
employed_by VARCHAR(20),
children INTEGER,
family INTEGER,
marital_status VARCHAR(20),
position VARCHAR(20),
income FLOAT,
income_Type VARCHAR(20),
housing VARCHAR(20),
house_ownership BOOLEAN)
""")
conn.commit()

In [4]:
#Заполняем таблицу для анкет необходимыми данными из каждого excel-файла
for file_name in os.listdir("forms"):
    full_path = os.path.join("forms", file_name)
    if os.path.isfile(full_path):
        wb = openpyxl.load_workbook(full_path)
        sheet = wb[wb.sheetnames[0]]
        
        id_number = int(sheet.cell(6, 1).value)
        
        birth_date = sheet.cell(4, 7).value
        if not(isinstance(birth_date, datetime.date) or isinstance(birth_date, datetime.datetime)):
            birth_date = datetime.datetime.strptime(birth_date, "%m.%d.%Y")
        else:
            birth_date = datetime.date(birth_date.year, birth_date.day, birth_date.month)
        
            
        gender = sheet.cell(4, 8).value

        gender = True if gender == 'Female' else False
        
        employed_by = sheet.cell(4, 9).value
        
        children = sheet.cell(8, 5).value
        
        family = sheet.cell(8, 6).value
        
        marital_status = sheet.cell(8, 7).value

        position = sheet.cell(10, 13).value
        
        income = sheet.cell(15, 1).value
    
        income_type = sheet.cell(17, 1).value

        housing = sheet.cell(19, 1).value

        house_ownership = sheet.cell(19, 5).value
           
        cursor.execute("""
        INSERT INTO Applications VALUES (%s, '%s', %s, '%s', %s, %s, '%s', '%s', %s, '%s', '%s', '%s')
        """ % (id_number, birth_date, gender, employed_by, children, family, 
               marital_status, position, income, income_type, housing, house_ownership))
conn.commit()

In [5]:
#Создаём пустую таблицу для контрактов с необходимыми нам колонками
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS Contracts 
(id_number INT, 
amount INT, 
term_month INT, 
contract_number INT, 
contract_type VARCHAR(50), 
annuity INT, 
contract_date DATE,
PRIMARY KEY (id_number),
FOREIGN KEY (id_number) REFERENCES Applications(id_number)) """)

conn.commit()

In [6]:
#Заполняем таблицу для контрактов необходимыми данными из каждого excel-файла
cursor = conn.cursor()

for file_name in os.listdir("contracts"):
    full_path = os.path.join("contracts", file_name)
    if os.path.isfile(full_path):
        wb = openpyxl.load_workbook(full_path)
        sheet = wb[wb.sheetnames[0]]
        
        id_number = int(sheet.cell(6, 1).value)
        
        amount = sheet.cell(8, 1).value if sheet.cell(8, 1).value else 'NULL'
        
        term = sheet.cell(10, 1).value if sheet.cell(10, 1).value else 'NULL'
        
        contract_number = int(sheet.cell(6, 5).value)
        
        type_c = sheet.cell(8, 5).value if sheet.cell(8, 5).value else 'NULL'
        
        annuity = sheet.cell(10, 5).value if sheet.cell(10, 5).value else 'NULL'

        contract_date = sheet.cell(2, 6).value
        if not(isinstance(contract_date, datetime.date) or isinstance(contract_date, datetime.datetime)):
            contract_date = datetime.datetime.strptime(contract_date, "%m.%d.%Y")
        else:
            contract_date = datetime.date(contract_date.year, contract_date.day, contract_date.month)

        try:
            cursor.execute("""
            INSERT INTO Contracts VALUES (%s, %s, %s, %s, '%s', %s, '%s')
            """ % (id_number, amount, term, contract_number, type_c, annuity, contract_date))
        except:
            print("ERROR! There are no application with id_number=%d!" % id_number)

conn.commit()

Мы получили готовую базу данных с имеющимися в файлах данными. Далее будем работать непосредственно с этими значениями.

### 2. Проверить корректность значений

Сначала проверим корректность данных в базе данных анкет.

In [7]:
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM Applications WHERE children < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с children < 0")
    cursor.execute("UPDATE Applications SET children=-1*(children) WHERE children < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с children < 0")


    
cursor.execute("SELECT COUNT(*) FROM Applications WHERE family < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с family < 0")
    cursor.execute("UPDATE Applications SET family=-1*(family) WHERE family < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с family < 0")
    
    

cursor.execute("SELECT COUNT(*) FROM Applications WHERE income < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с income < 0")
    cursor.execute("UPDATE Applications SET income=-1*(income) WHERE income < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с income < 0")
    
conn.commit()

Не найдены значения с children < 0
Не найдены значения с family < 0
Не найдены значения с income < 0


In [8]:
print("Проверка на единство обозначений работы")
cursor.execute("SELECT COUNT(employed_by), employed_by FROM Applications GROUP BY employed_by")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Проверка на единство обозначений семейного положения")
cursor.execute("SELECT COUNT(marital_status), marital_status FROM Applications GROUP BY marital_status")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)


print("Проверка на единство обозначений занимаемой должности") 
cursor.execute("SELECT COUNT(position), position FROM Applications GROUP BY position")
for i in cursor.fetchall():
    print(i)  
cursor.execute("""UPDATE Applications SET position = 'NULL' WHERE position = '<undefined>'""")
print("'<undefined>' заменено на NULL")
print("_" * 100)

print("Проверка на единство обозначений типов дохода")
cursor.execute("SELECT COUNT(income_type), income_type FROM Applications GROUP BY income_type")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Проверка на единство обозначений типа жилья")
cursor.execute("SELECT COUNT(housing), housing FROM Applications GROUP BY housing")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Проверка на единство обозначений владения домом")
cursor.execute("SELECT COUNT(house_ownership), house_ownership FROM Applications GROUP BY house_ownership")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

conn.commit()

Проверка на единство обозначений работы
(3, 'Business Entity Type 2')
(12, 'Business Entity Type 3')
(1, 'Construction')
(1, 'Electricity')
(1, 'Emergency')
(3, 'Government')
(1, 'Housing')
(1, 'Industry: type 1')
(2, 'Industry: type 11')
(1, 'Kindergarten')
(2, 'Medicine')
(1, 'Military')
(3, 'Other')
(1, 'Religion')
(2, 'School')
(1, 'Security')
(1, 'Security Ministries')
(8, 'Self-employed')
(2, 'Services')
(1, 'Trade: type 2')
(1, 'Trade: type 7')
(2, 'Transport: type 2')
(2, 'Transport: type 4')
(1, 'University')
(9, 'XNA')
____________________________________________________________________________________________________
Проверка на единство обозначений семейного положения
(8, 'Civil marriage')
(39, 'Married')
(1, 'Separated')
(12, 'Single / not married')
(3, 'Widow')
____________________________________________________________________________________________________
Проверка на единство обозначений занимаемой должности
(14, '<undefined>')
(2, 'Accountants')
(1, 'Cleaning staff'

В таблице отсуствуют отрицательные значения, есть разумное разбиение переменных. Проверим корректность в базе контрактов.

In [9]:
cursor.execute("SELECT COUNT(*) FROM contracts WHERE amount < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения amount < 0")
    cursor.execute("UPDATE contracts SET amount=-1*(amount) WHERE amount < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены отрицательные значения amount < 0")
    
cursor.execute("SELECT COUNT(*) FROM contracts WHERE term_month < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения term_month < 0")
    cursor.execute("UPDATE contracts SET term_month=-1*(term_month) WHERE term_month < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены отрицательные значения term_month < 0")
    
cursor.execute("SELECT COUNT(*) FROM contracts WHERE annuity < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения annuity < 0")
    cursor.execute("UPDATE contracts SET annuity=-1*(annuity) WHERE annuity < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены отрицательные значения annuity < 0")

Не найдены отрицательные значения amount < 0
Не найдены отрицательные значения term_month < 0
Не найдены отрицательные значения annuity < 0


In [10]:
print("Проверка на единство обозначений типов контракта")
cursor.execute("SELECT COUNT(contract_type), contract_type FROM contracts GROUP BY contract_type")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)


Проверка на единство обозначений типов контракта
(52, 'Cash loans')
(9, 'Revolving loans')
____________________________________________________________________________________________________


### 3. Создание словарей

In [11]:
#Кодирование для базы анкет
cursor.execute("UPDATE Applications SET employed_by = '0' WHERE employed_by = 'Business Entity Type 2'")
cursor.execute("UPDATE Applications SET employed_by = '1' WHERE employed_by = 'Business Entity Type 3'")
cursor.execute("UPDATE Applications SET employed_by = '2' WHERE employed_by = 'Construction'")
cursor.execute("UPDATE Applications SET employed_by = '3' WHERE employed_by = 'Electricity'")
cursor.execute("UPDATE Applications SET employed_by = '4' WHERE employed_by = 'Emergency'")
cursor.execute("UPDATE Applications SET employed_by = '5' WHERE employed_by = 'Government'")
cursor.execute("UPDATE Applications SET employed_by = '6' WHERE employed_by = 'Housing'")
cursor.execute("UPDATE Applications SET employed_by = '7' WHERE employed_by = 'Industry: type 1'")
cursor.execute("UPDATE Applications SET employed_by = '8' WHERE employed_by = 'Industry: type 11'")
cursor.execute("UPDATE Applications SET employed_by = '9' WHERE employed_by = 'Kindergarten'")
cursor.execute("UPDATE Applications SET employed_by = '10' WHERE employed_by = 'Medicine'")
cursor.execute("UPDATE Applications SET employed_by = '11' WHERE employed_by = 'Military'")
cursor.execute("UPDATE Applications SET employed_by = '12' WHERE employed_by = 'Other'")
cursor.execute("UPDATE Applications SET employed_by = '13' WHERE employed_by = 'Religion'")
cursor.execute("UPDATE Applications SET employed_by = '14' WHERE employed_by = 'School'")
cursor.execute("UPDATE Applications SET employed_by = '15' WHERE employed_by = 'Security'")
cursor.execute("UPDATE Applications SET employed_by = '16' WHERE employed_by = 'Security Ministries'")
cursor.execute("UPDATE Applications SET employed_by = '17' WHERE employed_by = 'Self-employed'")
cursor.execute("UPDATE Applications SET employed_by = '18' WHERE employed_by = 'Services'")
cursor.execute("UPDATE Applications SET employed_by = '19' WHERE employed_by = 'Trade: type 2'")
cursor.execute("UPDATE Applications SET employed_by = '20' WHERE employed_by = 'Trade: type 7'")
cursor.execute("UPDATE Applications SET employed_by = '21' WHERE employed_by = 'Transport: type 2'")
cursor.execute("UPDATE Applications SET employed_by = '22' WHERE employed_by = 'Transport: type 4'")
cursor.execute("UPDATE Applications SET employed_by = '23' WHERE employed_by = 'University'")
cursor.execute("UPDATE Applications SET employed_by = '24' WHERE employed_by = 'XNA'")


cursor.execute("UPDATE Applications SET marital_status = '0' WHERE marital_status = 'Single / not married'")
cursor.execute("UPDATE Applications SET marital_status = '1' WHERE marital_status = 'Married'")
cursor.execute("UPDATE Applications SET marital_status = '2' WHERE marital_status = 'Civil marriage'")
cursor.execute("UPDATE Applications SET marital_status = '3' WHERE marital_status = 'Separated'")
cursor.execute("UPDATE Applications SET marital_status = '4' WHERE marital_status = 'Widow'")

cursor.execute("UPDATE Applications SET position = '0' WHERE position = 'NULL'")
cursor.execute("UPDATE Applications SET position = '1' WHERE position = 'Accountants'")
cursor.execute("UPDATE Applications SET position = '2' WHERE position = 'Cleaning staff'")
cursor.execute("UPDATE Applications SET position = '3' WHERE position = 'Cooking staff'")
cursor.execute("UPDATE Applications SET position = '4' WHERE position = 'Core staff'")
cursor.execute("UPDATE Applications SET position = '5' WHERE position = 'Drivers'")
cursor.execute("UPDATE Applications SET position = '6' WHERE position = 'Laborers'")
cursor.execute("UPDATE Applications SET position = '7' WHERE position = 'Managers'")
cursor.execute("UPDATE Applications SET position = '8' WHERE position = 'Medicine staff'")
cursor.execute("UPDATE Applications SET position = '9' WHERE position = 'Private service staff'")
cursor.execute("UPDATE Applications SET position = '10' WHERE position = 'Sales staff'")
cursor.execute("UPDATE Applications SET position = '11' WHERE position = 'Security staff'")

cursor.execute("UPDATE Applications SET income_type = '0' WHERE income_type = 'Pensioner'")
cursor.execute("UPDATE Applications SET income_type = '1' WHERE income_type = 'Working'")
cursor.execute("UPDATE Applications SET income_type = '2' WHERE income_type = 'State servant'")
cursor.execute("UPDATE Applications SET income_type = '3' WHERE income_type = 'Commercial associate'")

cursor.execute("UPDATE Applications SET housing = '0' WHERE housing = 'With parents'")
cursor.execute("UPDATE Applications SET housing = '1' WHERE housing = 'House / apartment'")
cursor.execute("UPDATE Applications SET housing = '2' WHERE housing = 'Municipal apartment'")
cursor.execute("UPDATE Applications SET housing = '3' WHERE housing = 'Rented apartment'")

cursor.execute("UPDATE Applications SET house_ownership = '0' WHERE house_ownership = 'N'")
cursor.execute("UPDATE Applications SET house_ownership = '1' WHERE house_ownership = 'Y'")

conn.commit()

In [12]:
#Кодирование для базы контрактов (единсвтенный столбец с текстовой информацией)
cursor.execute("UPDATE Contracts SET contract_type = '0' WHERE contract_type = 'Revolving loans'")
cursor.execute("UPDATE Contracts SET contract_type = '1' WHERE contract_type = 'Cash loans'")
conn.commit()

In [13]:
#Создаем таблицу для расшифровки ключей
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS keys_applications 
(var VARCHAR(20), 
var_key VARCHAR(20), 
var_number INT) """)
conn.commit()

In [14]:
cursor.execute("""
        INSERT INTO keys_applications VALUES ('employed_by', 'Business Entity Type 2', 0), 
        ('employed_by', 'Business Entity Type 3', 1),
        ('employed_by', 'Construction', 2),
        ('employed_by', 'Electricity', 3),
        ('employed_by', 'Emergency', 4),
        ('employed_by', 'Government', 5),
        ('employed_by', 'Housing', 6),
        ('employed_by', 'Industry: type 1', 7),
        ('employed_by', 'Industry: type 11', 8),
        ('employed_by', 'Kindergarten', 9),
        ('employed_by', 'Medicine', 10),
        ('employed_by', 'Military', 11),
        ('employed_by', 'Other', 12),
        ('employed_by', 'Religion', 13),
        ('employed_by', 'School', 14),
        ('employed_by', 'Security', 15),
        ('employed_by', 'Security Ministries', 16),
        ('employed_by', 'Self-employed', 17),
        ('employed_by', 'Services', 18),
        ('employed_by', 'Trade: type 2', 19),
        ('employed_by', 'rade: type 7', 20),
        ('employed_by', 'Transport: type 2', 21),
        ('employed_by', 'Transport: type 4', 22),
        ('employed_by', 'University', 23),
        ('employed_by', 'XNA', 24),
        ('marital_status', 'Single / not married', 0),
        ('marital_status', 'Married', 1),
        ('marital_status', 'Civil marriage', 2),
        ('marital_status', 'Separated', 3),
        ('marital_status', 'Single / Widow', 4),
        ('position', 'NULL', 0),
        ('position', 'Accountants', 1),
        ('position', 'Cleaning staff', 2),
        ('position', 'Cooking staff', 3),
        ('position', 'Core staff', 4),
        ('position', 'Drivers', 5),
        ('position', 'Laborers', 6),
        ('position', 'Managers', 7),
        ('position', 'Medicine staff', 8),
        ('position', 'Private service staff', 9),
        ('position', 'Sales staff', 10),
        ('position', 'Security staff', 11),
        ('income_type', 'Pensioner', 0),
        ('income_type', 'Working', 1),
        ('income_type', 'State servant', 2),
        ('income_type', 'Commercial associate', 3),
        ('housing', 'With parents', 0),
        ('housing', 'House / apartment', 1),
        ('housing', 'Municipal apartment', 2),
        ('housing', 'Rented apartment', 3),
        ('house_ownership', 'N', 0),
        ('house_ownership', 'Y', 1),
        ('contract_type', 'Revolving loans', 0),
        ('contract_type', 'Cash loans', 1) """)

conn.commit()